In [2]:
import random
import pandas as pd
import numpy as np
import torch
import os


In [4]:
paper_data = pd.read_csv('../Data/cleaned_data.csv')
paper_data = paper_data.set_index('time (UTC)')
paper_data['Time'] = paper_data.index.astype(int)
paper_data

,Avg_Chloro,Nitrate,Phosphate,Silicate,Nitrite,SURF_TEMP_C,BOT_TEMP_C,SURF_SAL_PSU,BOT_SAL_PSU,SURF_DENS_kgm3,BOT_DENS_kgm3,AVG_TEMP_C,AVG_SAL_PSU,AVG_DENS_kgm3,Time
time (UTC),,,,,,,,,,,,,,,
0.0,1.82,0.43,0.24,5.6,0.03,20.3,15.9,33.77,33.62,1023.638094,1024.620655,18.10,33.695,1024.129374,0
7.0,2.71,0.83,0.37,5.1,0.04,17.8,16.9,33.66,33.62,1024.186379,1024.391719,17.35,33.640,1024.289049,7
13.0,0.99,0.45,0.24,3.5,0.00,22.2,18.3,33.74,33.58,1023.100557,1024.026040,20.25,33.660,1023.563298,13
20.0,1.23,0.34,0.17,4.2,0.00,19.8,19.4,33.68,33.62,1023.701088,1023.780535,9.90,33.650,1023.740812,20
28.0,2.41,0.29,0.12,3.8,0.00,22.5,22.5,33.73,33.69,1023.009067,1023.000725,11.25,33.710,1023.004896,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789.0,3.22,0.85,0.30,6.2,0.00,17.4,17.4,33.48,33.46,1024.146195,1024.152950,17.40,33.470,1024.149572,5789
5796.0,3.39,0.45,0.25,4.9,0.00,17.9,17.9,33.48,33.47,1024.026335,1024.040616,17.90,33.475,1024.033476,5796
5803.0,7.82,0.00,0.17,5.2,0.00,18.3,16.4,33.54,33.55,1023.974093,1024.454252,17.35,33.545,1024.214172,5803


In [15]:
def create_embeddings(data, E, tau, target=None):
    """
    Create time delay embeddings from time series data.
    
    Parameters:
    data (DataFrame): Input time series data where columns are features
    E (int): Maximum embedding dimension (number of time steps back)
    tau (int): Time delay between steps (absolute value will be used)
    target (str): Name of the target column in data
    
    Returns:
    X (list): List of embedded vectors, where each vector contains lagged values
    y (list): List of target values corresponding to each embedding
    """
    # Input validation
    if not isinstance(data, pd.DataFrame):
        raise TypeError("data must be a pandas DataFrame")
    if target not in data.columns:
        raise ValueError(f"target column '{target}' not found in data")
    if E < 1:
        raise ValueError("E must be at least 1")
    
    X = []
    y = []
    
    # Get the target column and ensure it's a numpy array
    target_values = data[target].values
    
    # Use absolute value of tau to ensure positive time delay
    tau = abs(tau)
    
    # Calculate the total length of history needed
    history_needed = (E - 1) * tau
    
    # Create embeddings for each viable time point
    for i in range(len(data)):
        # Skip points that don't have enough history
        if i < history_needed:
            continue
            
        # Create the embedding vector for this time point
        embedding = []
        
        # Add lagged values
        for e in range(E):
            # Calculate the index to look back to
            lag_idx = i - (e * tau)
            if lag_idx < 0:  # Safety check
                continue
            embedding.append(target_values[lag_idx])
        
        # Only add complete embeddings
        if len(embedding) == E:
            X.append(embedding)
            # Add the target value (next value in the series)
            if i < len(data) - 1 and (target is not None):  # Ensure we have a next value
                y.append(target_values[i + 1])
    
    # Convert to numpy arrays for convenience
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [16]:
X, y = create_embeddings(paper_data,3,-1,'Avg_Chloro')
X[0]

array([0.99, 2.71, 1.82])

In [ ]:
def total_embeddings(data, E, tau, target):
    X, y = create_embeddings(data, E, tau, col)
    for col in data.columns:
        if col != target:
            X_cur, _ = create_embeddings(data, E, tau, col)


    

        
